# Connect Databricks with Azure Data Lake Storage Gen2 with Sas Token

In [ ]:
spark.conf.set("fs.azure.account.auth.type.project1storagedatalake.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.project1storagedatalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.project1storagedatalake.dfs.core.windows.net", "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-11-30T01:46:19Z&st=2024-11-29T17:46:19Z&spr=https&sig=QM5FiBym63hm7ciklHZm2TAxi4rngzhsp4W1jd8vV7o%3D")

# DATA LOADING

## Reading Data 

In [ ]:
df_calendar = spark.read.format('csv')\
            .option('header', True)\
            .option("inferSchema", True)\
            .load('abfss://bronze@project1storagedatalake.dfs.core.windows.net/AdventureWorks_Calendar')


In [ ]:
df_customer = spark.read.format('csv')\
            .option('header', True)\
            .option("inferSchema", True)\
            .load('abfss://bronze@project1storagedatalake.dfs.core.windows.net/AdventureWorks_Customers')

In [ ]:
df_product_categories = spark.read.format('csv')\
            .option('header', True)\
            .option("inferSchema", True)\
            .load('abfss://bronze@project1storagedatalake.dfs.core.windows.net/AdventureWorks_Product_Categories')

In [ ]:
df_product_subcategories = spark.read.format('csv')\
            .option('header', True)\
            .option("inferSchema", True)\
            .load('abfss://bronze@project1storagedatalake.dfs.core.windows.net/AdventureWorks_Product_Subcategories')

In [ ]:
df_products = spark.read.format('csv')\
            .option('header', True)\
            .option("inferSchema", True)\
            .load('abfss://bronze@project1storagedatalake.dfs.core.windows.net/AdventureWorks_Products')

In [ ]:
df_returns = spark.read.format('csv')\
            .option('header', True)\
            .option("inferSchema", True)\
            .load('abfss://bronze@project1storagedatalake.dfs.core.windows.net/AdventureWorks_Returns')

In [ ]:
df_sales = spark.read.format('csv')\
            .option('header', True)\
            .option("inferSchema", True)\
            .load('abfss://bronze@project1storagedatalake.dfs.core.windows.net/AdventureWorks_Sales*')

In [ ]:
df_territories = spark.read.format('csv')\
            .option('header', True)\
            .option("inferSchema", True)\
            .load('abfss://bronze@project1storagedatalake.dfs.core.windows.net/AdventureWorks_Territories')

# TRANSFORMATIONS

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Calendar

In [ ]:
df_calendar = df_calendar.withColumn('Month', month(col('Date')))\
                         .withColumn('Year', year(col('Date')))\
                         .withColumn('Week', weekofyear(col('Date')))\
                         .withColumn('Day', dayofmonth(col('Date')))\
                         .withColumn('DayofWeek', dayofweek(col('Date')))\
                         .withColumn('WeekOfYear', weekofyear(col('Date')))\
                         .withColumn('Quarter', quarter(col('Date')))

df_calendar.display()

In [ ]:
df_calendar.write.format('parquet')\
                .mode('append')\
                .option('path', 'abfss://silver@project1storagedatalake.dfs.core.windows.net/Calendar')\
                .save()

### Customer

In [ ]:
df_customer = df_customer.withColumn('FullName', concat_ws(' ', df_customer.FirstName, df_customer.LastName))\
                        .withColumn('Gender',when(df_customer.Gender == 'M', 'Male').when(df_customer.Gender == 'F', 'Female').otherwise(df_customer.Gender))\
                        .withColumn("AnnualIncome",regexp_replace(df_customer.AnnualIncome, "[$,]", "").cast("int"))\
                        .withColumn("Age",floor(datediff(current_date(), to_date(df_customer.BirthDate, 'MM/dd/yyyy')) / 365.25))
                        

In [ ]:
df_customer.write.format('parquet')\
                .mode('append')\
                .option('path', 'abfss://silver@project1storagedatalake.dfs.core.windows.net/Customer')\
                .save()

### Product Categories

In [ ]:
df_product_categories.write.format('parquet')\
                .mode('append')\
                .option('path', 'abfss://silver@project1storagedatalake.dfs.core.windows.net/ProductCategories')\
                .save()

### Product Subcategories

In [ ]:
df_product_subcategories.write.format('parquet')\
                .mode('append')\
                .option('path', 'abfss://silver@project1storagedatalake.dfs.core.windows.net/ProductSubcategories')\
                .save()

### Products  

In [ ]:
df_products = df_products.withColumn('ProductSKU', split(df_products.ProductSKU, '-')[0])

df_products.display()   

In [ ]:
df_products.write.format('parquet')\
                .mode('append')\
                .option('path', 'abfss://silver@project1storagedatalake.dfs.core.windows.net/Products')\
                .save()

### Returns

In [ ]:
df_returns.write.format('parquet')\
                .mode('append')\
                .option('path', 'abfss://silver@project1storagedatalake.dfs.core.windows.net/Returns')\
                .save()

### Territories

In [ ]:
df_territories.write.format('parquet')\
                .mode('append')\
                .option('path', 'abfss://silver@project1storagedatalake.dfs.core.windows.net/Territories')\
                .save()

### Sales 

In [ ]:
df_sales = df_sales.withColumn('StockDate',to_timestamp('StockDate'))\
                        .withColumn('OrderNumber', regexp_replace('OrderNumber', 'S','T'))\
                        .withColumn('Multiply',col('OrderLineItem')*col('OrderQuantity'))

In [ ]:
df_sales.write.format('parquet')\
                .mode('append')\
                .option('path', 'abfss://silver@project1storagedatalake.dfs.core.windows.net/Sales')\
                .save()

# Analysis

In [ ]:
df_sales.groupBy('OrderDate').agg(count('OrderNumber').alias('Total Orders')).display() 

In [ ]:
df_product_categories.display()

In [ ]:
df_territories.display()   